## Step 2: Augment database with asset data view and supporting metadata

#### For all elements/assets with PIPoints, create a default Data View having as columns all those PIPoints

Data View column names are the attribute names in AF, so elements sharing a template have identical Data Views modulo missing streams

This notebook also creates specialized Data Views:

* Some are subsets of the default Data View, per asset
* Others are multi-assets version of the above  

#### After running this notebook, the graph has all the necessary information for Step 2: projection of tags/metadata on streams and Data View creation 

In [1]:
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport
import json
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
import yaml
import pprint
from collections import OrderedDict

## Input Parameters

In [2]:
config_file = "config-delmar.yaml"
# config_file = "config-cmu2.yaml"
# config_file = "config-windfarm.yaml"  # 
# config_file = "config-acad-prod-desc-v2.yaml"

In [3]:
with open(config_file) as f:
    config = yaml.safe_load(f)
# config

In [4]:
# Input parameters

database_name = config["db"]["database_name"]
root_element = config["db"]["root_element"]
ocs_asset_db = dv_id_prefix = config["ocs"]["configuration"]["asset_db"]

# Each assets in a Hub dataset is endowed with default Data View (DV) "All_Columns" having all the asset streams
# The next line of code defines a dictionary to build all default (DV)
# If <column list> is empty list [], include all streams in Data View
dv_defs = {"Default": ("all_columns", [], lambda id: True)}
has_no_dv = lambda id: False


# YAML config file may embed additional code to define additional custom dataviews
try:
    exec(config["custom_dvs"])
except:
    pass

dv_defs

{'Default': ('all_columns', [], <function __main__.<lambda>(id)>)}

In [5]:
sample_transport = RequestsHTTPTransport(
    url=config["graphql"]["endpoint"], verify=False, retries=3
)
client = Client(transport=sample_transport, fetch_schema_from_transport=True)

In [6]:
db_query = gql(
    """
query DatabaseId($database: String) {
    Database(name: $database) {
        name
        id
        asset_db
    }
}
"""
)
print(f"database_name={database_name}")
db = client.execute(db_query, variable_values={"database": database_name})
print(json.dumps(db, indent=4))

database_name=Pilot_Plant
{
    "Database": [
        {
            "name": "Pilot_Plant",
            "id": "pilot.plant:F1RDcFWpYyj-GkWKwoVLgdSfpwG1XyxiSvX0WMNyDlsRHq8AUElBRi1BQ0FEXFBJTE9UUExBTlQ",
            "asset_db": "pilot.plant"
        }
    ]
}


In [7]:
db_id = db["Database"][0]["id"]
asset_db = db["Database"][0]["asset_db"]
asset_db, db_id

('pilot.plant',
 'pilot.plant:F1RDcFWpYyj-GkWKwoVLgdSfpwG1XyxiSvX0WMNyDlsRHq8AUElBRi1BQ0FEXFBJTE9UUExBTlQ')

In [8]:
root_query = gql(
    """
query RootElementTree($root : String, $asset_db: String) {
    Element(name: $root, asset_db: $asset_db) {
        name
        id
        has_element(orderBy: name_asc) {
            name
            id
            all_streams(orderBy: name_asc) {
                name
                stream_name
                id
                pointsource
            }
        }
    }
}
"""
)
j = client.execute(
    root_query, variable_values={"root": root_element, "asset_db": ocs_asset_db}
)
print(json.dumps(j, indent=4))

{
    "Element": [
        {
            "name": "Pilot Plant",
            "id": "pilot.plant:F1EmcFWpYyj-GkWKwoVLgdSfpw7j7_VF8k6xGpdwANOjT1RQUElBRi1BQ0FEXFBJTE9UUExBTlRcUExBTlRcUElMT1QgUExBTlQ",
            "has_element": [
                {
                    "name": "HOT-1",
                    "id": "pilot.plant:F1EmcFWpYyj-GkWKwoVLgdSfpwBj__VF8k6xGpdwANOjT1RQUElBRi1BQ0FEXFBJTE9UUExBTlRcUExBTlRcUElMT1QgUExBTlRcSE9ULTE",
                    "all_streams": [
                        {
                            "name": "E-102 Duty",
                            "stream_name": "delmar.DELMAR_UNIT1_E-102_DUTY",
                            "id": "F1DPpjXwPKxU8EWdWQR_TqpF6QwS8AAAQUNBRC1QSURBLVZNMFxERUxNQVIuREVMTUFSX1VOSVQxX0UtMTAyX0RVVFk",
                            "pointsource": "PIC"
                        },
                        {
                            "name": "E-102 Heat Transfer Coefficient",
                            "stream_name": "delmar.DELMAR_UNIT1_E-102_HTC",
  

In [9]:
elements = j["Element"][0]["has_element"] 
len(elements), elements

(2,
 [{'name': 'HOT-1',
   'id': 'pilot.plant:F1EmcFWpYyj-GkWKwoVLgdSfpwBj__VF8k6xGpdwANOjT1RQUElBRi1BQ0FEXFBJTE9UUExBTlRcUExBTlRcUElMT1QgUExBTlRcSE9ULTE',
   'all_streams': [{'name': 'E-102 Duty',
     'stream_name': 'delmar.DELMAR_UNIT1_E-102_DUTY',
     'id': 'F1DPpjXwPKxU8EWdWQR_TqpF6QwS8AAAQUNBRC1QSURBLVZNMFxERUxNQVIuREVMTUFSX1VOSVQxX0UtMTAyX0RVVFk',
     'pointsource': 'PIC'},
    {'name': 'E-102 Heat Transfer Coefficient',
     'stream_name': 'delmar.DELMAR_UNIT1_E-102_HTC',
     'id': 'F1DPpjXwPKxU8EWdWQR_TqpF6QtC8AAAQUNBRC1QSURBLVZNMFxERUxNQVIuREVMTUFSX1VOSVQxX0UtMTAyX0hUQw',
     'pointsource': 'PIC'},
    {'name': 'E-102 LMTD',
     'stream_name': 'delmar.DELMAR_UNIT1_E-102_LMTD',
     'id': 'F1DPpjXwPKxU8EWdWQR_TqpF6QzS8AAAQUNBRC1QSURBLVZNMFxERUxNQVIuREVMTUFSX1VOSVQxX0UtMTAyX0xNVEQ',
     'pointsource': 'PIC'},
    {'name': 'Heater Charge Flow',
     'stream_name': 'delmar.DELMAR_UNIT1_FI-102.PV',
     'id': 'F1DPpjXwPKxU8EWdWQR_TqpF6QyC8AAAQUNBRC1QSURBLVZNMFxERUxNQVIuREVMT

In [10]:
all_elements = elements
for element in elements:
    j = client.execute(root_query, variable_values={"root": element["name"]})
    all_elements += j["Element"][0]["has_element"] 
all_elements

[{'name': 'HOT-1',
  'id': 'pilot.plant:F1EmcFWpYyj-GkWKwoVLgdSfpwBj__VF8k6xGpdwANOjT1RQUElBRi1BQ0FEXFBJTE9UUExBTlRcUExBTlRcUElMT1QgUExBTlRcSE9ULTE',
  'all_streams': [{'name': 'E-102 Duty',
    'stream_name': 'delmar.DELMAR_UNIT1_E-102_DUTY',
    'id': 'F1DPpjXwPKxU8EWdWQR_TqpF6QwS8AAAQUNBRC1QSURBLVZNMFxERUxNQVIuREVMTUFSX1VOSVQxX0UtMTAyX0RVVFk',
    'pointsource': 'PIC'},
   {'name': 'E-102 Heat Transfer Coefficient',
    'stream_name': 'delmar.DELMAR_UNIT1_E-102_HTC',
    'id': 'F1DPpjXwPKxU8EWdWQR_TqpF6QtC8AAAQUNBRC1QSURBLVZNMFxERUxNQVIuREVMTUFSX1VOSVQxX0UtMTAyX0hUQw',
    'pointsource': 'PIC'},
   {'name': 'E-102 LMTD',
    'stream_name': 'delmar.DELMAR_UNIT1_E-102_LMTD',
    'id': 'F1DPpjXwPKxU8EWdWQR_TqpF6QzS8AAAQUNBRC1QSURBLVZNMFxERUxNQVIuREVMTUFSX1VOSVQxX0UtMTAyX0xNVEQ',
    'pointsource': 'PIC'},
   {'name': 'Heater Charge Flow',
    'stream_name': 'delmar.DELMAR_UNIT1_FI-102.PV',
    'id': 'F1DPpjXwPKxU8EWdWQR_TqpF6QyC8AAAQUNBRC1QSURBLVZNMFxERUxNQVIuREVMTUFSX1VOSVQxX0ZJLTEwMi

In [11]:
len(all_elements), all_elements

(14,
 [{'name': 'HOT-1',
   'id': 'pilot.plant:F1EmcFWpYyj-GkWKwoVLgdSfpwBj__VF8k6xGpdwANOjT1RQUElBRi1BQ0FEXFBJTE9UUExBTlRcUExBTlRcUElMT1QgUExBTlRcSE9ULTE',
   'all_streams': [{'name': 'E-102 Duty',
     'stream_name': 'delmar.DELMAR_UNIT1_E-102_DUTY',
     'id': 'F1DPpjXwPKxU8EWdWQR_TqpF6QwS8AAAQUNBRC1QSURBLVZNMFxERUxNQVIuREVMTUFSX1VOSVQxX0UtMTAyX0RVVFk',
     'pointsource': 'PIC'},
    {'name': 'E-102 Heat Transfer Coefficient',
     'stream_name': 'delmar.DELMAR_UNIT1_E-102_HTC',
     'id': 'F1DPpjXwPKxU8EWdWQR_TqpF6QtC8AAAQUNBRC1QSURBLVZNMFxERUxNQVIuREVMTUFSX1VOSVQxX0UtMTAyX0hUQw',
     'pointsource': 'PIC'},
    {'name': 'E-102 LMTD',
     'stream_name': 'delmar.DELMAR_UNIT1_E-102_LMTD',
     'id': 'F1DPpjXwPKxU8EWdWQR_TqpF6QzS8AAAQUNBRC1QSURBLVZNMFxERUxNQVIuREVMTUFSX1VOSVQxX0UtMTAyX0xNVEQ',
     'pointsource': 'PIC'},
    {'name': 'Heater Charge Flow',
     'stream_name': 'delmar.DELMAR_UNIT1_FI-102.PV',
     'id': 'F1DPpjXwPKxU8EWdWQR_TqpF6QyC8AAAQUNBRC1QSURBLVZNMFxERUxNQVIuREVM

In [12]:
def asset_with_dv(x):
    return True


try:
    exec(config["asset_with_dv"])
    print("ok")
except:
    pass

elements = [e for e in all_elements if asset_with_dv(e["name"])]
len(elements), elements

ok
asset: HOT-1 DV: True
asset: HOT-3 DV: True
asset: E-101 DV: False
asset: E-102 DV: False
asset: HTR-101 DV: False
asset: TK-101 DV: False
asset: TK-102 DV: False
asset: TK-103 DV: False
asset: Bottoms Product DV: False
asset: Energy In DV: False
asset: Energy Out DV: False
asset: Overhead Product DV: False
asset: Pressure DV: False
asset: T-200 DV: False


(2,
 [{'name': 'HOT-1',
   'id': 'pilot.plant:F1EmcFWpYyj-GkWKwoVLgdSfpwBj__VF8k6xGpdwANOjT1RQUElBRi1BQ0FEXFBJTE9UUExBTlRcUExBTlRcUElMT1QgUExBTlRcSE9ULTE',
   'all_streams': [{'name': 'E-102 Duty',
     'stream_name': 'delmar.DELMAR_UNIT1_E-102_DUTY',
     'id': 'F1DPpjXwPKxU8EWdWQR_TqpF6QwS8AAAQUNBRC1QSURBLVZNMFxERUxNQVIuREVMTUFSX1VOSVQxX0UtMTAyX0RVVFk',
     'pointsource': 'PIC'},
    {'name': 'E-102 Heat Transfer Coefficient',
     'stream_name': 'delmar.DELMAR_UNIT1_E-102_HTC',
     'id': 'F1DPpjXwPKxU8EWdWQR_TqpF6QtC8AAAQUNBRC1QSURBLVZNMFxERUxNQVIuREVMTUFSX1VOSVQxX0UtMTAyX0hUQw',
     'pointsource': 'PIC'},
    {'name': 'E-102 LMTD',
     'stream_name': 'delmar.DELMAR_UNIT1_E-102_LMTD',
     'id': 'F1DPpjXwPKxU8EWdWQR_TqpF6QzS8AAAQUNBRC1QSURBLVZNMFxERUxNQVIuREVMTUFSX1VOSVQxX0UtMTAyX0xNVEQ',
     'pointsource': 'PIC'},
    {'name': 'Heater Charge Flow',
     'stream_name': 'delmar.DELMAR_UNIT1_FI-102.PV',
     'id': 'F1DPpjXwPKxU8EWdWQR_TqpF6QyC8AAAQUNBRC1QSURBLVZNMFxERUxNQVIuREVMT

In [13]:
len(set([i["name"] for i in elements]))

2

In [14]:
d = {}
for e in elements:
    d[e["name"]] = e
d.keys()

dict_keys(['HOT-1', 'HOT-3'])

In [15]:
elements = [d[i] for i in sorted([i for i in d])]
len(elements), elements

(2,
 [{'name': 'HOT-1',
   'id': 'pilot.plant:F1EmcFWpYyj-GkWKwoVLgdSfpwBj__VF8k6xGpdwANOjT1RQUElBRi1BQ0FEXFBJTE9UUExBTlRcUExBTlRcUElMT1QgUExBTlRcSE9ULTE',
   'all_streams': [{'name': 'E-102 Duty',
     'stream_name': 'delmar.DELMAR_UNIT1_E-102_DUTY',
     'id': 'F1DPpjXwPKxU8EWdWQR_TqpF6QwS8AAAQUNBRC1QSURBLVZNMFxERUxNQVIuREVMTUFSX1VOSVQxX0UtMTAyX0RVVFk',
     'pointsource': 'PIC'},
    {'name': 'E-102 Heat Transfer Coefficient',
     'stream_name': 'delmar.DELMAR_UNIT1_E-102_HTC',
     'id': 'F1DPpjXwPKxU8EWdWQR_TqpF6QtC8AAAQUNBRC1QSURBLVZNMFxERUxNQVIuREVMTUFSX1VOSVQxX0UtMTAyX0hUQw',
     'pointsource': 'PIC'},
    {'name': 'E-102 LMTD',
     'stream_name': 'delmar.DELMAR_UNIT1_E-102_LMTD',
     'id': 'F1DPpjXwPKxU8EWdWQR_TqpF6QzS8AAAQUNBRC1QSURBLVZNMFxERUxNQVIuREVMTUFSX1VOSVQxX0UtMTAyX0xNVEQ',
     'pointsource': 'PIC'},
    {'name': 'Heater Charge Flow',
     'stream_name': 'delmar.DELMAR_UNIT1_FI-102.PV',
     'id': 'F1DPpjXwPKxU8EWdWQR_TqpF6QyC8AAAQUNBRC1QSURBLVZNMFxERUxNQVIuREVMT

In [16]:
# Virtual attributes creation
# NOTE: role of virtual attributes is to copy all child element dynamic attributes for which the parent element doesn't
#   have a similar attribute referencing the pipoint. This way all attributes from parent (union of real and virtual)
#   constitute a strict cover of all the pipoints under it. If a pipoint has multiple children referencing, the first one
#   (in element name alphabetical order) is picked for the virtual attribute creation

stream_query = gql(
    """
query Element($name: String!) {
    Element(name: $name) {
        name
        id
        has_attribute(filter: { is_stream: true }) {
          name
          id
          has_dynamic {
            name
            id
            asset_id
          }
        }
        has_element(orderBy: name_asc) {
          name
          id
          has_attribute(filter: { is_stream: true }) {
            name
            id
            has_dynamic {
              name 
              id
              asset_id
            }
          }
        }
    }
}
"""
)

# result = client.execute(stream_query, variable_values={"name": "HOT-1"})
# result

In [17]:
def attr_to_clone(elem, seen_d, debug=False):
    print(f"...processing {elem['name']}...") if debug else ()
    to_clone = []
    for k, i in enumerate(elem["has_attribute"], 1):
        j = i["has_dynamic"][0]
        if not seen_d.get(j["id"], None):
            # print(k, i["id"], i["name"], j["id"]) if debug else ()
            print(k, i["name"], j["id"]) if debug else ()
            seen_d[j["id"]] = elem["name"]
            to_clone += [i["id"]]
        else:
            print(
                "$$$ exist:",
                seen_d[j["id"]],
                j["name"],
            ) if debug else ()
    return to_clone

In [18]:
attr_query = gql(
    """
query Attribute($id: ID!) {
  attr: Attribute(id: $id) {
    af_template
    asset_db
    id
    name
    is_stream
    type
    virtual
    parent
    has_dynamic {
        id
    }
  }
}
"""
)

vattr_mutation = gql(
    """
mutation VirtualAttribute(
    $af_template: String!
    $asset_db: String!
    $id: ID!
    $name: String!
    $is_stream: Boolean!
    $type: String!
    $virtual: Boolean!
    $parent: String!
    $pid: ID!
) {
    vattr: MergeAttribute(
        af_template: $af_template
        asset_db: $asset_db
        id: $id
        name: $name
        is_stream: $is_stream
        type: $type
        virtual: $virtual
        parent: $parent
    ) {
        id
    }
    dlink: MergeAttributeHas_dynamic(
        from: {id: $id}
        to: {id: $pid}
    ) {
        from {
            name
            id
        }
        to {
            name
            id
        }
    }
}
"""
)


def clone_attribute_as_virtual(asset_id, ids_to_clone):
    for attr_id in ids_to_clone:
        r = client.execute(attr_query, variable_values={"id": attr_id})
        attr = r["attr"][0]
        s = client.execute(
            vattr_mutation,
            variable_values={
                "af_template": attr["af_template"],
                "asset_db": attr["asset_db"],
                "id": attr["id"] + ":virtual",
                "name": attr["name"],
                "is_stream": attr["is_stream"],
                "type": attr["type"],
                "virtual": True,
                "parent": asset_id,
                "pid": attr["has_dynamic"][0]["id"],
            },
        )

In [19]:
for asset_id in d.keys():
    result = client.execute(stream_query, variable_values={"name": asset_id})
    for e in result["Element"]:
        seen = {}
        ok_ids = attr_to_clone(e, seen)
        ids_to_clone = []
        for i in e["has_element"]:
            ids_to_clone += attr_to_clone(i, seen)
        print(e["name"], len(ids_to_clone), ids_to_clone)
        clone_attribute_as_virtual(asset_id, ids_to_clone)

HOT-1 8 ['pilot.plant:F1AbEcFWpYyj-GkWKwoVLgdSfpwFT__VF8k6xGpdwANOjT1RQ-CCsRRTBRUG_MnZwiOh8CAUElBRi1BQ0FEXFBJTE9UUExBTlRcUExBTlRcUElMT1QgUExBTlRcSE9ULTFcRS0xMDF8RS0xMDEgUlVOIFNUQVRVUw', 'pilot.plant:F1AbEcFWpYyj-GkWKwoVLgdSfpwEj__VF8k6xGpdwANOjT1RQ9MV-t8hWiku0fyTZ8qpGhwUElBRi1BQ0FEXFBJTE9UUExBTlRcUExBTlRcUElMT1QgUExBTlRcSE9ULTFcRS0xMDJ8RS0xMDIgVFVCRSBTSURFIE9VVExFVCBURU1QRVJBVFVSRSBDT05UUk9MTEVSIE1PREU', 'pilot.plant:F1AbEcFWpYyj-GkWKwoVLgdSfpwEj__VF8k6xGpdwANOjT1RQ5pEj7oSLR0qYb5b063LzOwUElBRi1BQ0FEXFBJTE9UUExBTlRcUExBTlRcUElMT1QgUExBTlRcSE9ULTFcRS0xMDJ8RS0xMDIgTE1URA', 'pilot.plant:F1AbEcFWpYyj-GkWKwoVLgdSfpwEj__VF8k6xGpdwANOjT1RQ5jGXC9s9k0mDinZlGBpzwQUElBRi1BQ0FEXFBJTE9UUExBTlRcUExBTlRcUElMT1QgUExBTlRcSE9ULTFcRS0xMDJ8RS0xMDIgSEVBVCBUUkFOU0ZFUiBDT0VGRklDSUVOVA', 'pilot.plant:F1AbEcFWpYyj-GkWKwoVLgdSfpwEj__VF8k6xGpdwANOjT1RQaSB1Dmmtike-HMbbe4ni8gUElBRi1BQ0FEXFBJTE9UUExBTlRcUExBTlRcUElMT1QgUExBTlRcSE9ULTFcRS0xMDJ8RS0xMDIgRFVUWQ', 'pilot.plant:F1AbEcFWpYyj-GkWKwoVLgdSfpwDD__VF8k6xGpdwANO

In [20]:
elements[0]

{'name': 'HOT-1',
 'id': 'pilot.plant:F1EmcFWpYyj-GkWKwoVLgdSfpwBj__VF8k6xGpdwANOjT1RQUElBRi1BQ0FEXFBJTE9UUExBTlRcUExBTlRcUElMT1QgUExBTlRcSE9ULTE',
 'all_streams': [{'name': 'E-102 Duty',
   'stream_name': 'delmar.DELMAR_UNIT1_E-102_DUTY',
   'id': 'F1DPpjXwPKxU8EWdWQR_TqpF6QwS8AAAQUNBRC1QSURBLVZNMFxERUxNQVIuREVMTUFSX1VOSVQxX0UtMTAyX0RVVFk',
   'pointsource': 'PIC'},
  {'name': 'E-102 Heat Transfer Coefficient',
   'stream_name': 'delmar.DELMAR_UNIT1_E-102_HTC',
   'id': 'F1DPpjXwPKxU8EWdWQR_TqpF6QtC8AAAQUNBRC1QSURBLVZNMFxERUxNQVIuREVMTUFSX1VOSVQxX0UtMTAyX0hUQw',
   'pointsource': 'PIC'},
  {'name': 'E-102 LMTD',
   'stream_name': 'delmar.DELMAR_UNIT1_E-102_LMTD',
   'id': 'F1DPpjXwPKxU8EWdWQR_TqpF6QzS8AAAQUNBRC1QSURBLVZNMFxERUxNQVIuREVMTUFSX1VOSVQxX0UtMTAyX0xNVEQ',
   'pointsource': 'PIC'},
  {'name': 'Heater Charge Flow',
   'stream_name': 'delmar.DELMAR_UNIT1_FI-102.PV',
   'id': 'F1DPpjXwPKxU8EWdWQR_TqpF6QyC8AAAQUNBRC1QSURBLVZNMFxERUxNQVIuREVMTUFSX1VOSVQxX0ZJLTEwMi5QVg',
   'points

In [21]:
elements_info = [
    (
        elements[i]["name"],
        elements[i]["id"],
        [
            stream
            for stream in elements[i]["all_streams"]
            # if j["pointsource"] != "AZURE"
        ],
    )
    for i in range(len(elements))
    # if len(elements[i]["has_attribute"]) > 0
]
elements_info

[('HOT-1',
  'pilot.plant:F1EmcFWpYyj-GkWKwoVLgdSfpwBj__VF8k6xGpdwANOjT1RQUElBRi1BQ0FEXFBJTE9UUExBTlRcUExBTlRcUElMT1QgUExBTlRcSE9ULTE',
  [{'name': 'E-102 Duty',
    'stream_name': 'delmar.DELMAR_UNIT1_E-102_DUTY',
    'id': 'F1DPpjXwPKxU8EWdWQR_TqpF6QwS8AAAQUNBRC1QSURBLVZNMFxERUxNQVIuREVMTUFSX1VOSVQxX0UtMTAyX0RVVFk',
    'pointsource': 'PIC'},
   {'name': 'E-102 Heat Transfer Coefficient',
    'stream_name': 'delmar.DELMAR_UNIT1_E-102_HTC',
    'id': 'F1DPpjXwPKxU8EWdWQR_TqpF6QtC8AAAQUNBRC1QSURBLVZNMFxERUxNQVIuREVMTUFSX1VOSVQxX0UtMTAyX0hUQw',
    'pointsource': 'PIC'},
   {'name': 'E-102 LMTD',
    'stream_name': 'delmar.DELMAR_UNIT1_E-102_LMTD',
    'id': 'F1DPpjXwPKxU8EWdWQR_TqpF6QzS8AAAQUNBRC1QSURBLVZNMFxERUxNQVIuREVMTUFSX1VOSVQxX0UtMTAyX0xNVEQ',
    'pointsource': 'PIC'},
   {'name': 'Heater Charge Flow',
    'stream_name': 'delmar.DELMAR_UNIT1_FI-102.PV',
    'id': 'F1DPpjXwPKxU8EWdWQR_TqpF6QyC8AAAQUNBRC1QSURBLVZNMFxERUxNQVIuREVMTUFSX1VOSVQxX0ZJLTEwMi5QVg',
    'pointsource': 'PI

In [22]:
for element, element_id, _ in elements_info:
    mutation = gql(
        """
mutation AssetWithDataView($from: _DatabaseInput!, $to: _ElementInput!) {
    MergeDatabaseAsset_with_dv(
        from: $from,
        to: $to
    ) {
        from {
            name
        }
        to {
            name
        }
    }
}
"""
    )
    # pprint.pprint(mutation)
    reply = client.execute(
        mutation, variable_values={"from": {"id": db_id}, "to": {"id": element_id}}
    )
    print(json.dumps(reply, indent=4))

{
    "MergeDatabaseAsset_with_dv": {
        "from": {
            "name": "Pilot_Plant"
        },
        "to": {
            "name": "HOT-1"
        }
    }
}
{
    "MergeDatabaseAsset_with_dv": {
        "from": {
            "name": "Pilot_Plant"
        },
        "to": {
            "name": "HOT-3"
        }
    }
}


In [23]:
def dataview_streams_transaction(dv_id, dv_streams, asset_id):
    if len(dv_streams) == 0:
        print(f"@@ dv_id={dv_id}")
        return
    mutations = []
    for i, stream in enumerate(dv_streams):
        stream_mutation = """
stream{0}: 
    AddDataViewHas_stream(
        from: {{id: "{1}" }}
        to: {{id: "{2}" }}
    ) {{
        from {{
            name
        }}
        to {{
            name
        }}
    }}   
    """.format(
            i, dv_id, stream["id"]
        )
        # print("mutation", i, stream_mutation)
        mutations.append(stream_mutation)

    if asset_id:
        for i, stream in enumerate(dv_streams):
            stream_mutation = """          
stream_asset{0}:
    MergePIPoint(
        id: "{1}"
        asset_id: "HOT-1"
    ) {{
            id
            asset_id
    }}
    """.format(
                i, stream["id"]
            )
            mutations.append(stream_mutation)

    dv_stream_mutation = gql(
        "mutation dataview_streams {\n" + "".join(mutations) + "\n}"
    )

    reply = client.execute(dv_stream_mutation)
    print(f"[add-dv-streams: {dv_id} ({len(json.dumps(reply, indent=4))})]", end="")

In [24]:
mutation_dv_node = gql(
    """
mutation AssetDataView(
            $asset_db: String!, 
            $asset_id: [String]!, 
            $columns: String!, 
            $description: String, 
            $id: ID!, 
            $name: String!, 
            $ocs_tag: String!) {
    MergeDataView(
        asset_db: $asset_db
        asset_id: $asset_id
        id: $id
        columns: $columns 
        description: $description
        name: $name
        ocs_sync: false
        ocs_tag: $ocs_tag
    ) {
        name
        id 
    }
}
    """
)


mutation_dv_rel = gql(
    """
mutation ElementDataView($from: _ElementInput!, $to: _DataViewInput!) {
    MergeElementHas_dataview(
        from: $from
        to: $to
    ) {
        from {
            name
        }
        to {
            name
            id
        }
    }
}
    """
)


def dataview_id(asset_ids, ocs_tag, multiple=None):
    dv_asset = (
        multiple
        if len(asset_ids) > 1
        else asset_ids[0].lower().replace(" ", ".").replace("/", "__")
    )
    dv_id = f"{dv_id_prefix}-{dv_asset}"
    if ocs_tag != "all_columns":
        dv_id = f"{dv_id}-{ocs_tag}"
    return dv_id


def create_asset_dataview(
    dv_name,
    asset_ids,
    elem_ids,
    all_streams,
    ocs_tag,
    columns,
    ocs_column_key="column_name",
    multiple=None,
):
    dv_id = dataview_id(asset_ids, ocs_tag, multiple)
    asset_desc = (
        f"Asset {asset_ids[0]}" if len(asset_ids) == 1 else f"Assets {multiple.upper()}"
    )
    reply_node = client.execute(
        mutation_dv_node,
        variable_values={
            "asset_db": asset_db,
            "asset_id": asset_ids,
            "columns": str(sorted(columns))
            if len(columns) > 0
            else str(sorted(set([s["name"] for s in all_streams]))),
            "description": f"Hub DV for {asset_desc} - {dv_name}",
            "id": dv_id,
            "name": dv_name,
            "ocs_tag": ocs_tag,
            "ocs_column_key": ocs_column_key,
        },
    )

    for elem_id in elem_ids:
        reply_rel = client.execute(
            mutation_dv_rel,
            variable_values={"from": {"id": elem_id}, "to": {"id": dv_id}},
        )
        # print(json.dumps(reply_rel, indent=4))

    if len(columns) == 0:
        dv_streams = all_streams
    else:
        dv_streams = [s for s in all_streams if s["name"] in columns]

    asset_id = asset_ids[0] if len(asset_ids) == 1 else None
    dataview_streams_transaction(dv_id, dv_streams, asset_id)


element_ids = {}

for dv_name in dv_defs.keys():
    print(f"dv_name={dv_name}")
    ocs_tag, columns, has_custom_dv = dv_defs[dv_name]
    for asset_id, elem_id, all_streams in elements_info:
        if has_no_dv(asset_id):
            continue
        element_ids[asset_id] = elem_id
        # no specialized data view except for fermenter vessels
        if len(columns) > 0 and not has_custom_dv(asset_id):
            continue
        # print(dv_name, [asset_id], all_streams, ocs_tag, columns)
        create_asset_dataview(
            dv_name, [asset_id], [elem_id], all_streams, ocs_tag, columns
        )

dv_name=Default
[add-dv-streams: pilot.plant-hot-1 (12180)][add-dv-streams: pilot.plant-hot-3 (13054)]

In [25]:
def extract_columns(asset_ids, ocs_tag):
    query = gql(
        """
    query DataView($id: ID!) {
        DataView(id: $id) {
            columns
      }  
    }
    """
    )
    reply = client.execute(
        query, variable_values={"id": dataview_id([asset_ids[0]], ocs_tag)}
    )
    # print(dataview_id([asset_ids[0]], ocs_tag), reply["DataView"][0]["columns"])
    return sorted(
        [
            i.strip()
            for i in reply["DataView"][0]["columns"][1:-1].replace("'", "").split(",")
        ]
    )


multi_asset_dvs = []
try:
    exec(config["multi_asset_dvs"])
except:
    pass
print(config["multi_asset_dvs"])

for asset_ids, dv_suffix in multi_asset_dvs:
    for dv_name in dv_defs.keys():
        ocs_tag, columns, has_custom_dv = dv_defs[dv_name]
        if len(columns) == 0:
            columns = extract_columns(asset_ids, ocs_tag)
        elem_ids = [element_ids[i] for i in asset_ids]
        create_asset_dataview(
            dv_name + "+", asset_ids, elem_ids, [], ocs_tag, columns, multiple=dv_suffix
        )

# No multi-asset DV



### Asset metadata gathering and update, plus geo-data if applicable

In [26]:
print("db name:", database_name)
meta_query = gql(
    """
query Database($name: String) {
  Database(name: $name) {
    asset_with_dv(orderBy: name_asc) {
      name
      id
      has_attribute(orderBy: name_asc, filter: {is_stream: false}) {
        name
        value
        type
      }
      has_element {
        name
        has_attribute(filter: { is_stream: false }) {
          name
          value
          type
        }
      }
    }
  }
}
"""
)
result = client.execute(meta_query, variable_values={"name": database_name})
result

db name: Pilot_Plant


{'Database': [{'asset_with_dv': [{'name': 'HOT-1',
     'id': 'pilot.plant:F1EmcFWpYyj-GkWKwoVLgdSfpwBj__VF8k6xGpdwANOjT1RQUElBRi1BQ0FEXFBJTE9UUExBTlRcUExBTlRcUElMT1QgUExBTlRcSE9ULTE',
     'has_attribute': [],
     'has_element': [{'name': 'TK-103', 'has_attribute': []},
      {'name': 'TK-102', 'has_attribute': []},
      {'name': 'TK-101', 'has_attribute': []},
      {'name': 'HTR-101', 'has_attribute': []},
      {'name': 'E-102',
       'has_attribute': [{'name': 'Heat Exchange Area',
         'value': '23',
         'type': 'Double'}]},
      {'name': 'E-101', 'has_attribute': []}]},
    {'name': 'HOT-3',
     'id': 'pilot.plant:F1EmcFWpYyj-GkWKwoVLgdSfpw8T7_VF8k6xGpdwANOjT1RQUElBRi1BQ0FEXFBJTE9UUExBTlRcUExBTlRcUElMT1QgUExBTlRcSE9ULTM',
     'has_attribute': [],
     'has_element': [{'name': 'T-200', 'has_attribute': []},
      {'name': 'Pressure', 'has_attribute': []},
      {'name': 'Overhead Product', 'has_attribute': []},
      {'name': 'Energy Out', 'has_attribute': []},
   

In [27]:
def convert2value(s, vtype):
    if vtype == "Double":
        return float(s)
    else:
        return str(s)


def extract_meta(js, sub=False):
    j = js["has_attribute"]
    prefix = "" if not sub else (js["name"].upper() + ".")
    return {f"{prefix}{k['name']}": convert2value(k["value"], k["type"]) for k in j}

In [28]:
# test
d = extract_meta(result["Database"][0]["asset_with_dv"][0])
d

{}

In [29]:
meta_mutation = gql(
    """
mutation UpdateMeta($id: ID!, $meta: String) {
  MergeElement(id: $id, asset_metadata: $meta) {
    id
    name
    asset_metadata
  }
}
"""
)

geo_mutation = gql(
    """
mutation UpdateGeo($id: ID!, $lat: Float, $long: Float)  {
    MergeElement(id: $id, latitude: $lat, longitude: $long, location: {latitude: $lat, longitude: $long}) {
        name
        id
        latitude
        longitude
        location {
            latitude
            longitude
        }
    }
}    
"""
)


def update_meta(wid, meta):
    result = client.execute(
        meta_mutation, variable_values={"id": wid, "meta": str(meta)}
    )
    return result


def update_geo(wid, lat, long):
    result = client.execute(
        geo_mutation, variable_values={"id": wid, "lat": lat, "long": long}
    )
    return result

In [30]:
nb = len(result["Database"][0]["asset_with_dv"])
for i in range(nb):
    dai = result["Database"][0]["asset_with_dv"][i]
    d = extract_meta(dai)
    for j in range(len(dai.get("has_element", []))):
        d.update(extract_meta(dai["has_element"][j], True))
    print(dai["name"], dai["id"], d)
    update_meta(dai["id"], d)
    if config["db"].get("asset_geodata", False):
        update_geo(dai["id"], d.get("Latitude", 0.0), d.get("Longitude", 0.0))

HOT-1 pilot.plant:F1EmcFWpYyj-GkWKwoVLgdSfpwBj__VF8k6xGpdwANOjT1RQUElBRi1BQ0FEXFBJTE9UUExBTlRcUExBTlRcUElMT1QgUExBTlRcSE9ULTE {'E-102.Heat Exchange Area': 23.0}
HOT-3 pilot.plant:F1EmcFWpYyj-GkWKwoVLgdSfpw8T7_VF8k6xGpdwANOjT1RQUElBRi1BQ0FEXFBJTE9UUExBTlRcUExBTlRcUElMT1QgUExBTlRcSE9ULTM {}
